In [ ]:
# SISO 5G gNB-UE Simulation using AIRSTRAN D 2200

# Import or install Sionna
try:
    import sionna.rt
except ImportError as e:
    import os
    os.system("pip install sionna-rt")
    import sionna.rt

# Other imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import mitsuba as mi
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", message="invalid value encountered in multiply")
warnings.filterwarnings("ignore", category=UserWarning, module="jupyter_client")

# Import relevant components from Sionna RT
from sionna.rt import load_scene, Transmitter, Receiver, Transmitter, Camera, PathSolver
from sionna.rt import AntennaArray, PlanarArray, SceneObject, ITURadioMaterial
from sionna.rt.antenna_pattern import antenna_pattern_registry

# Load empty scene
scene = load_scene("/home/tingjunlab/Development/geo2sigmap/scenes/Duke/scene.xml")

In [ ]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath('../src'))

In [ ]:
# Install plyfile if not available
try:
    from plyfile import PlyData
except ImportError:
    import os
    os.system("pip install plyfile")
    from plyfile import PlyData

In [ ]:
# ============================================
# SISO Configuration: gNB to UE
# ============================================

# Set the operating frequency (n48 band for 5G)
scene.frequency = 3.7e9  # 3.7 GHz

# Define UE position (fixed to start)
ue_position = [10.0, 0.0, 0.0]   # UE position (x, y, z in meters)

# ============================================
# Antenna Configuration
# ============================================

# gNB antenna: 3GPP TR 38.901 pattern (AIRSTRAN D 2200)
gnb_pattern_factory = antenna_pattern_registry.get("tr38901")
gnb_pattern = gnb_pattern_factory(polarization="V")

# UE antenna: Isotropic pattern (typical for mobile devices)
# This will be required for matching the calculations of the RadioMapSolver()
ue_pattern_factory = antenna_pattern_registry.get("iso")
# Polarization should also match the transmitter
ue_pattern = ue_pattern_factory(polarization="V")

# SISO: Single antenna element at origin [0, 0, 0] for both TX and RX
single_element = np.array([[0.0, 0.0, 0.0]])  # Shape: (1, 3)

# Configure antenna arrays
scene.tx_array = AntennaArray(
    antenna_pattern=gnb_pattern,
    normalized_positions=single_element.T  # Shape: (3, 1)
)

scene.rx_array = AntennaArray(
    antenna_pattern=ue_pattern,
    normalized_positions=single_element.T  # Shape: (3, 1)
)

# ============================================
# Add Receiver to Scene
# ============================================

# Create UE receiver
rx = Receiver(name="ue", position=ue_position, display_radius=0.03)
scene.add(rx)

# ============================================
# Configure Propagation Environment
# ============================================

# Disable scattering for basic simulation
for radio_material in scene.radio_materials.values():
    radio_material.scattering_coefficient = 0.0

In [ ]:
from scene_parser import extract_building_info
from tx_placement import TxPlacement

In [ ]:
# ============================================
# Place gNB on a Specific Building
# ============================================

scene_xml_path = "/home/tingjunlab/Development/geo2sigmap/scenes/Duke/scene.xml"
building_info = extract_building_info(scene_xml_path, verbose=True)

# Old: 46
selected_building_id = 46  # Change this to your desired building number

# TxPlacement will create the transmitter if it doesn't exist and place it on the building
# Correct parameter order: (scene, tx_name, scene_xml_path, building_id, offset)
tx_placer = TxPlacement(scene, "gnb", scene_xml_path, selected_building_id, offset=5.0)
tx_placer.set_rooftop_center()

# Get reference to the transmitter (already added to scene by TxPlacement)
tx = tx_placer.tx
# Convert to flat numpy array instead of nested list
gnb_position = tx.position.numpy().flatten().tolist()

# Point antenna toward UE
tx.look_at(ue_position)

print(f"\nSuccess! gNB placed on building {selected_building_id}")
print(f"Position: {gnb_position}")

# ============================================
# Compute Propagation Paths
# ============================================

# Instantiate path solver
p_solver = PathSolver()

# Compute propagation paths
paths = p_solver(
    scene=scene,
    max_depth=5,
    los=True,
    specular_reflection=True,
    diffuse_reflection=False,
    diffraction=True,
    edge_diffraction=True,
    refraction=False,
    seed=41
)

# ============================================
# Visualize Scene
# ============================================

# Setup camera
cam = Camera(position=(100.0, 100.0, 50.0))
cam.look_at(gnb_position)

# Preview the scene with propagation paths
scene.preview(
    paths=paths,
    resolution=[1000, 1000],
    clip_at=50,
    show_orientations=True
)

gnb_pattern.show();

In [ ]:
from boresight_pathsolver import (
    optimize_boresight_pathsolver,
    create_zone_mask
)
import matplotlib.pyplot as plt

# ============================================
# NEW BINARY MASK APPROACH
# ============================================
# Instead of creating a target radiomap with specific power levels,
# we create a BINARY MASK that defines the coverage zone.
# The optimizer will maximize coverage within this zone.

# Map configuration
map_config = {
    'center': [gnb_position[0], gnb_position[1], 0],
    'size': [1400, 1400],
    'cell_size': (1.0, 1.0),
    'ground_height': 0.0,
}

# ============================================
# OPTION 1: Angular Sector Zone (RECOMMENDED)
# ============================================
# Create a wedge-shaped sector pointing from the TX
zone_mask, naive_look_at, zone_stats = create_zone_mask(
    map_config=map_config,
    zone_type='box',
    origin_point=gnb_position,
    #zone_params={
    #    'angle_start': 10,
    #    'angle_end': 10+65,      # Try 65° to match TR38.901 beamwidth
    #    'radius': 600         # Larger radius
    #},
    zone_params={
        'center': [600.0, 200.0],
        'width': 300,      
        'height': 300         
    },
    target_height=0.0,
    scene_xml_path="/home/tingjunlab/Development/geo2sigmap/scenes/Duke/scene.xml",
    exclude_buildings=True           # ← This is True by default
)

print("=" * 70)
print("ZONE MASK CREATED")
print("=" * 70)
print(f"Zone type: Angular sector (0° to 120°, radius 100m)")
print(f"Zone contains {zone_stats['num_cells']} grid cells")
print(f"Zone coverage: {zone_stats['coverage_fraction']*100:.1f}% of map")
print(f"Naive baseline look-at: {zone_stats['look_at_xyz']}")
print(f"Zone centroid: {zone_stats['centroid_xy']}")
print()

# Visualize the Zone Mask
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(
    zone_mask,
    origin='lower',
    cmap='RdYlGn',
    extent=[
        map_config['center'][0] - map_config['size'][0]/2,
        map_config['center'][0] + map_config['size'][0]/2,
        map_config['center'][1] - map_config['size'][1]/2,
        map_config['center'][1] + map_config['size'][1]/2,
    ],
    vmin=0,
    vmax=1
)
plt.colorbar(im, ax=ax, label='Zone Mask (1.0 = in zone, 0.0 = out)')
ax.plot(gnb_position[0], gnb_position[1], 'r*', markersize=20, label='TX')
ax.plot(zone_stats['centroid_xy'][0], zone_stats['centroid_xy'][1], 'bo',
        markersize=10, label='Zone Centroid')
ax.plot(zone_stats['look_at_xyz'][0], zone_stats['look_at_xyz'][1], 'g^',
        markersize=10, label='Naive Baseline')
ax.set_title(f'Coverage Zone Mask ({zone_stats["num_cells"]} cells)')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

# ============================================
# Run Optimization with Binary Mask
# ============================================
print("=" * 70)
print("STARTING BINARY MASK OPTIMIZATION")
print("=" * 70)

# Sanity check: verify mask at zone center
test_x, test_y = -100, 200  # Zone center
test_i = int((test_x - (map_config['center'][0] - map_config['size'][0] / 2)) / map_config['cell_size'][0])
test_j = int((test_y - (map_config['center'][1] - map_config['size'][1] / 2)) / map_config['cell_size'][1])
print(f"Zone center ({test_x}, {test_y}) maps to grid cell [{test_j}, {test_i}]")
print(f"Mask value at zone center: {zone_mask[test_j, test_i]}")
print(f"Expected: 1.0 (inside zone)")

# NEW: Returns angles [azimuth, elevation] instead of Cartesian coordinates
# NEW LOSS FUNCTION: Combines target coverage maximization + interference minimization
best_angles, loss_hist, angle_hist, grad_hist, cov_stats = optimize_boresight_pathsolver(
    scene=scene,
    tx_name="gnb",
    map_config=map_config,
    scene_xml_path=scene_xml_path,
    zone_mask=zone_mask,
    initial_boresight=naive_look_at.tolist(),  # Converted to angles internally
    num_sample_points=1000,
    learning_rate=.1,
    num_iterations=200,
    loss_type='coverage_maximize',  # Uses soft-min/soft-max with interference control
    use_grid_search_init=False,
    grid_search_params={
        'num_angular_samples': 40,  # Finer grid = better init
        'num_elevation_samples': 15,  # More elevation angles
        'num_sample_points': 250,   # Fast grid search
    },
    verbose=True,
    seed=22,
    save_radiomap_frames=False,
    frame_save_interval=10,
    output_dir="/home/tingjunlab/Development/optimize_tx/optimization_gif"
)

print(f"\nOptimization complete!")
print(f"Best angles: Azimuth={best_angles[0]:.1f}°, Elevation={best_angles[1]:.1f}°")
print(f"Final coverage stats: {cov_stats}")

In [ ]:
from boresight_pathsolver import compare_boresight_performance
from angle_utils import azimuth_elevation_to_yaw_pitch, compute_initial_angles_from_position
import mitsuba as mi

# Compute naive angles from the naive look_at position
tx = scene.get("gnb")
tx_position = [
    float(tx.position[0][0]),
    float(tx.position[1][0]),
    float(tx.position[2][0])
]
print(tx_position)

# Convert naive look_at position to angles
naive_azimuth, naive_elevation = compute_initial_angles_from_position(
    tx_position, naive_look_at.tolist()
)
naive_angles = [naive_azimuth, naive_elevation]

print(f"Naive baseline angles: Azimuth={naive_azimuth:.1f}°, Elevation={naive_elevation:.1f}°")
print(f"Optimized angles: Azimuth={best_angles[0]:.1f}°, Elevation={best_angles[1]:.1f}°")
print()

# Compare performance using angles
fig, comparison_stats = compare_boresight_performance(
    scene=scene,
    tx_name="gnb",
    map_config=map_config,
    zone_mask=zone_mask,
    naive_angles=naive_angles,
    optimized_angles=best_angles,
    title="Boresight Optimization Results (Angle-Based)"
)

plt.show()

In [ ]:
from sionna.rt import RadioMapSolver
from angle_utils import azimuth_elevation_to_yaw_pitch
import matplotlib.pyplot as plt
import mitsuba as mi

# Apply the optimized angles to the transmitter
tx = scene.get("gnb")
best_azimuth, best_elevation = best_angles
yaw_rad, pitch_rad = azimuth_elevation_to_yaw_pitch(best_azimuth, best_elevation)
tx.orientation = mi.Point3f(float(yaw_rad), float(pitch_rad), 0.0)

print("=" * 70)
print("RADIO MAP VISUALIZATION (OPTIMIZED)")
print("=" * 70)
print(f"Optimized angles: Azimuth={best_azimuth:.1f}°, Elevation={best_elevation:.1f}°")
print(f"TX location: {tx.position}")
print(f"TX orientation (yaw, pitch, roll): ({yaw_rad:.3f}, {pitch_rad:.3f}, 0.0) rad")
print(f"Coverage zone: {cov_stats['num_samples_in_zone']}/{cov_stats['num_samples_total']} samples")
print(f"Final loss: {cov_stats['final_loss']:.4f}")
print()

# Generate radio map matching the optimization configuration
rm_solver = RadioMapSolver()
rm = rm_solver(
    scene,
    max_depth=5,
    samples_per_tx=int(6e8),
    cell_size=map_config['cell_size'],
    center=map_config['center'],
    orientation=[0, 0, 0],
    size=map_config['size'],
    los=True,
    specular_reflection=True,
    diffuse_reflection=True,
    diffraction=True,
    edge_diffraction=True,
    refraction=False,
    stop_threshold=None,
)

# Extract signal levels
rss_watts = rm.rss.numpy()[0, :, :]
signal_strength_dBm = 10.0 * np.log10(rss_watts + 1e-30) + 30.0

# ============================================
# Analyze Coverage in the Zone
# ============================================
zone_power = signal_strength_dBm[zone_mask == 1.0]

print("Coverage Analysis:")
print(f"  Mean power in zone: {np.mean(zone_power):.2f} dBm")
print(f"  Median power in zone: {np.median(zone_power):.2f} dBm")
print(f"  Min power in zone: {np.min(zone_power):.2f} dBm")
print(f"  Max power in zone: {np.max(zone_power):.2f} dBm")
print(f"  Std dev in zone: {np.std(zone_power):.2f} dB")
print()

# Count cells above various thresholds
thresholds = [-90, -85, -80, -75, -70]
for thresh in thresholds:
    above = np.sum(zone_power > thresh)
    pct = 100.0 * above / len(zone_power)
    print(f"  Above {thresh} dBm: {above}/{len(zone_power)} ({pct:.1f}%)")
print()

# ============================================
# Visualize Results
# ============================================
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

extent = [
    map_config['center'][0] - map_config['size'][0]/2,
    map_config['center'][0] + map_config['size'][0]/2,
    map_config['center'][1] - map_config['size'][1]/2,
    map_config['center'][1] + map_config['size'][1]/2,
]

# Plot 1: Zone Mask
im1 = axes[0].imshow(zone_mask, origin='lower', cmap='RdYlGn', extent=extent, vmin=0, vmax=1)
axes[0].plot(gnb_position[0], gnb_position[1], 'r*', markersize=20, label='TX')
axes[0].set_title('Coverage Zone Mask')
axes[0].set_xlabel('X (m)')
axes[0].set_ylabel('Y (m)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
plt.colorbar(im1, ax=axes[0], label='Mask Value')

# Plot 2: RadioMap (Signal Strength)
im2 = axes[1].imshow(signal_strength_dBm, origin='lower', cmap='viridis', 
                     extent=extent, vmin=-120, vmax=-60)
axes[1].plot(gnb_position[0], gnb_position[1], 'r*', markersize=20, label='TX')
axes[1].set_title(f'Optimized Signal Strength\nAz={best_azimuth:.1f}°, El={best_elevation:.1f}°')
axes[1].set_xlabel('X (m)')
axes[1].set_ylabel('Y (m)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.colorbar(im2, ax=axes[1], label='Signal Strength (dBm)')

# Plot 3: Masked Coverage (only show zone)
masked_signal = np.where(zone_mask == 1.0, signal_strength_dBm, np.nan)
im3 = axes[2].imshow(masked_signal, origin='lower', cmap='viridis',
                     extent=extent, vmin=-120, vmax=-60)
axes[2].plot(gnb_position[0], gnb_position[1], 'r*', markersize=20, label='TX')
axes[2].set_title(f'Coverage in Zone Only\nMedian: {np.median(zone_power):.1f} dBm')
axes[2].set_xlabel('X (m)')
axes[2].set_ylabel('Y (m)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
plt.colorbar(im3, ax=axes[2], label='Signal Strength (dBm)')

plt.tight_layout()
plt.show()

# ============================================
# Optimization Diagnostics - UPDATED for Angles
# ============================================
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Loss history
axes[0, 0].plot(loss_hist, 'b-', linewidth=2)
axes[0, 0].set_title('Loss History')
axes[0, 0].set_xlabel('Iteration')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].grid(True, alpha=0.3)

# Gradient norms
axes[0, 1].plot(grad_hist, 'r-', linewidth=2)
axes[0, 1].set_title('Gradient Norm History')
axes[0, 1].set_xlabel('Iteration')
axes[0, 1].set_ylabel('Gradient Norm')
axes[0, 1].set_yscale('log')
axes[0, 1].grid(True, alpha=0.3)

# Angle trajectory - Azimuth
angle_arr = np.array(angle_hist)
axes[1, 0].plot(angle_arr[:, 0], 'g-', linewidth=2, label='Azimuth')
axes[1, 0].axhline(y=best_azimuth, color='b', linestyle='--', label=f'Final: {best_azimuth:.1f}°')
axes[1, 0].set_title('Azimuth Angle Optimization')
axes[1, 0].set_xlabel('Iteration')
axes[1, 0].set_ylabel('Azimuth (degrees)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Angle trajectory - Elevation
axes[1, 1].plot(angle_arr[:, 1], 'm-', linewidth=2, label='Elevation')
axes[1, 1].axhline(y=best_elevation, color='b', linestyle='--', label=f'Final: {best_elevation:.1f}°')
axes[1, 1].set_title('Elevation Angle Optimization')
axes[1, 1].set_xlabel('Iteration')
axes[1, 1].set_ylabel('Elevation (degrees)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()